In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_FV
from utils_training import FVTrainingData
from utils_training import evaluate_FVmodel

In [4]:
from behavior_model import FVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 4, 0.25
acc_std_dev = 0.07
Inputs, Inputs_Vl, Outputs, Weights = FVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_50_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Inputs_Vl_train = [Inputs_Vl[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Inputs_Vl_val = [Inputs_Vl[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_FV(Inputs_train, Inputs_Vl_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_FV(Inputs_val, Inputs_Vl_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = FVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_fv_val_50.pth'
best_train_model_path = f'.\\weights\\bc_fv_train_50.pth'
best_trainval_model_path = f'.\\weights\\bc_fv_trainval_50.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, inputs_vl, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        inputs_vl = inputs_vl.reshape(inputs_vl.shape[0],1,inputs_vl.shape[1])
        inputs_vl = inputs_vl.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs,inputs_vl)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_FVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_FVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 260.34,      Loss_min: 196.64, Loss_max: 319.39 
      Loss_mean_val: 258.82, Loss_min_val: 203.4,      Loss_max_val: 318.21 
      Loss_mean_trainval: 259.84
Epoch 2/4000, Loss_mean: 177.28,      Loss_min: 128.01, Loss_max: 273.86 
      Loss_mean_val: 173.33, Loss_min_val: 134.34,      Loss_max_val: 255.11 
      Loss_mean_trainval: 175.96
Epoch 3/4000, Loss_mean: 159.04,      Loss_min: 119.53, Loss_max: 209.7 
      Loss_mean_val: 162.87, Loss_min_val: 119.57,      Loss_max_val: 233.27 
      Loss_mean_trainval: 160.32
Epoch 4/4000, Loss_mean: 152.32,      Loss_min: 110.26, Loss_max: 212.3 
      Loss_mean_val: 150.21, Loss_min_val: 111.98,      Loss_max_val: 200.24 
      Loss_mean_trainval: 151.62
Epoch 5/4000, Loss_mean: 147.04,      Loss_min: 106.19, Loss_max: 196.01 
      Loss_mean_val: 147.06, Loss_min_val: 85.42,      Loss_max_val: 218.14 
      Loss_mean_trainval: 147.05
Epoch 6/4000, Loss_mean: 141.11,      Loss_min: 90.85, Loss_max: 192.44 
      

Epoch 47/4000, Loss_mean: 62.46,      Loss_min: 42.47, Loss_max: 85.91 
      Loss_mean_val: 67.73, Loss_min_val: 39.44,      Loss_max_val: 110.54 
      Loss_mean_trainval: 64.22
Epoch 48/4000, Loss_mean: 61.16,      Loss_min: 40.39, Loss_max: 81.68 
      Loss_mean_val: 66.13, Loss_min_val: 41.3,      Loss_max_val: 89.4 
      Loss_mean_trainval: 62.81
Epoch 49/4000, Loss_mean: 63.04,      Loss_min: 40.85, Loss_max: 93.44 
      Loss_mean_val: 67.71, Loss_min_val: 37.9,      Loss_max_val: 98.97 
      Loss_mean_trainval: 64.6
Epoch 50/4000, Loss_mean: 60.77,      Loss_min: 33.49, Loss_max: 84.24 
      Loss_mean_val: 67.11, Loss_min_val: 38.0,      Loss_max_val: 95.47 
      Loss_mean_trainval: 62.88
Epoch 51/4000, Loss_mean: 60.31,      Loss_min: 37.26, Loss_max: 100.35 
      Loss_mean_val: 64.12, Loss_min_val: 44.01,      Loss_max_val: 93.24 
      Loss_mean_trainval: 61.58
Epoch 52/4000, Loss_mean: 59.47,      Loss_min: 31.3, Loss_max: 82.19 
      Loss_mean_val: 63.94, Loss_min_

Epoch 93/4000, Loss_mean: 43.08,      Loss_min: 23.41, Loss_max: 61.48 
      Loss_mean_val: 48.75, Loss_min_val: 32.84,      Loss_max_val: 77.85 
      Loss_mean_trainval: 44.97
Epoch 94/4000, Loss_mean: 42.32,      Loss_min: 22.08, Loss_max: 55.35 
      Loss_mean_val: 47.57, Loss_min_val: 28.05,      Loss_max_val: 66.74 
      Loss_mean_trainval: 44.07
Epoch 95/4000, Loss_mean: 41.94,      Loss_min: 16.6, Loss_max: 66.25 
      Loss_mean_val: 47.6, Loss_min_val: 32.26,      Loss_max_val: 65.98 
      Loss_mean_trainval: 43.83
Epoch 96/4000, Loss_mean: 41.06,      Loss_min: 25.41, Loss_max: 59.38 
      Loss_mean_val: 46.16, Loss_min_val: 25.41,      Loss_max_val: 66.2 
      Loss_mean_trainval: 42.76
Epoch 97/4000, Loss_mean: 41.14,      Loss_min: 22.7, Loss_max: 61.08 
      Loss_mean_val: 47.1, Loss_min_val: 29.45,      Loss_max_val: 71.01 
      Loss_mean_trainval: 43.13
Epoch 98/4000, Loss_mean: 40.51,      Loss_min: 20.7, Loss_max: 56.86 
      Loss_mean_val: 46.83, Loss_min_va

Epoch 139/4000, Loss_mean: 31.79,      Loss_min: 21.42, Loss_max: 44.63 
      Loss_mean_val: 40.59, Loss_min_val: 15.38,      Loss_max_val: 74.92 
      Loss_mean_trainval: 34.73
Epoch 140/4000, Loss_mean: 31.73,      Loss_min: 17.99, Loss_max: 51.22 
      Loss_mean_val: 38.1, Loss_min_val: 21.33,      Loss_max_val: 56.56 
      Loss_mean_trainval: 33.86
Epoch 141/4000, Loss_mean: 31.71,      Loss_min: 13.67, Loss_max: 45.53 
      Loss_mean_val: 38.0, Loss_min_val: 21.63,      Loss_max_val: 60.42 
      Loss_mean_trainval: 33.81
Epoch 142/4000, Loss_mean: 31.13,      Loss_min: 18.84, Loss_max: 46.63 
      Loss_mean_val: 37.62, Loss_min_val: 22.4,      Loss_max_val: 50.29 
      Loss_mean_trainval: 33.29
Epoch 143/4000, Loss_mean: 31.4,      Loss_min: 18.59, Loss_max: 44.58 
      Loss_mean_val: 38.35, Loss_min_val: 22.01,      Loss_max_val: 50.98 
      Loss_mean_trainval: 33.72
Epoch 144/4000, Loss_mean: 30.73,      Loss_min: 19.63, Loss_max: 40.61 
      Loss_mean_val: 36.54, Los

Epoch 185/4000, Loss_mean: 25.02,      Loss_min: 12.01, Loss_max: 36.28 
      Loss_mean_val: 31.55, Loss_min_val: 16.88,      Loss_max_val: 48.53 
      Loss_mean_trainval: 27.19
Epoch 186/4000, Loss_mean: 24.47,      Loss_min: 14.48, Loss_max: 34.55 
      Loss_mean_val: 31.66, Loss_min_val: 18.2,      Loss_max_val: 47.84 
      Loss_mean_trainval: 26.87
Epoch 187/4000, Loss_mean: 24.89,      Loss_min: 10.97, Loss_max: 38.91 
      Loss_mean_val: 31.54, Loss_min_val: 19.47,      Loss_max_val: 43.02 
      Loss_mean_trainval: 27.11
Epoch 188/4000, Loss_mean: 24.7,      Loss_min: 14.0, Loss_max: 36.41 
      Loss_mean_val: 31.15, Loss_min_val: 13.8,      Loss_max_val: 46.61 
      Loss_mean_trainval: 26.85
Epoch 189/4000, Loss_mean: 24.73,      Loss_min: 14.47, Loss_max: 39.37 
      Loss_mean_val: 30.89, Loss_min_val: 15.72,      Loss_max_val: 45.3 
      Loss_mean_trainval: 26.79
Epoch 190/4000, Loss_mean: 24.21,      Loss_min: 14.15, Loss_max: 34.91 
      Loss_mean_val: 30.36, Loss

...

Epoch 3801/4000, Loss_mean: 1.78,      Loss_min: 0.53, Loss_max: 3.45 
      Loss_mean_val: 17.03, Loss_min_val: 7.97,      Loss_max_val: 36.47 
      Loss_mean_trainval: 6.87
Epoch 3802/4000, Loss_mean: 1.66,      Loss_min: 0.82, Loss_max: 2.46 
      Loss_mean_val: 15.75, Loss_min_val: 8.0,      Loss_max_val: 24.03 
      Loss_mean_trainval: 6.36
Epoch 3803/4000, Loss_mean: 1.73,      Loss_min: 0.75, Loss_max: 2.69 
      Loss_mean_val: 16.09, Loss_min_val: 5.23,      Loss_max_val: 42.19 
      Loss_mean_trainval: 6.52
Epoch 3804/4000, Loss_mean: 1.84,      Loss_min: 0.91, Loss_max: 2.79 
      Loss_mean_val: 15.71, Loss_min_val: 8.26,      Loss_max_val: 23.87 
      Loss_mean_trainval: 6.46
Epoch 3805/4000, Loss_mean: 1.93,      Loss_min: 0.94, Loss_max: 2.77 
      Loss_mean_val: 15.79, Loss_min_val: 4.51,      Loss_max_val: 32.33 
      Loss_mean_trainval: 6.56
Epoch 3806/4000, Loss_mean: 1.83,      Loss_min: 1.01, Loss_max: 3.01 
      Loss_mean_val: 15.57, Loss_min_val: 6.56,   

Epoch 3848/4000, Loss_mean: 1.64,      Loss_min: 0.84, Loss_max: 2.59 
      Loss_mean_val: 16.43, Loss_min_val: 5.52,      Loss_max_val: 32.06 
      Loss_mean_trainval: 6.57
Epoch 3849/4000, Loss_mean: 1.69,      Loss_min: 0.67, Loss_max: 2.93 
      Loss_mean_val: 15.99, Loss_min_val: 6.84,      Loss_max_val: 38.84 
      Loss_mean_trainval: 6.46
Epoch 3850/4000, Loss_mean: 1.76,      Loss_min: 0.8, Loss_max: 2.84 
      Loss_mean_val: 16.86, Loss_min_val: 7.14,      Loss_max_val: 36.5 
      Loss_mean_trainval: 6.8
Epoch 3851/4000, Loss_mean: 1.69,      Loss_min: 0.86, Loss_max: 2.55 
      Loss_mean_val: 15.82, Loss_min_val: 7.24,      Loss_max_val: 35.53 
      Loss_mean_trainval: 6.4
Epoch 3852/4000, Loss_mean: 1.66,      Loss_min: 0.62, Loss_max: 3.0 
      Loss_mean_val: 15.67, Loss_min_val: 9.32,      Loss_max_val: 25.14 
      Loss_mean_trainval: 6.33
Epoch 3853/4000, Loss_mean: 1.63,      Loss_min: 0.74, Loss_max: 2.72 
      Loss_mean_val: 15.57, Loss_min_val: 6.5,      Lo

Epoch 3895/4000, Loss_mean: 1.55,      Loss_min: 0.76, Loss_max: 2.44 
      Loss_mean_val: 16.16, Loss_min_val: 8.72,      Loss_max_val: 32.71 
      Loss_mean_trainval: 6.42
Epoch 3896/4000, Loss_mean: 1.56,      Loss_min: 0.77, Loss_max: 2.89 
      Loss_mean_val: 15.27, Loss_min_val: 7.22,      Loss_max_val: 30.68 
      Loss_mean_trainval: 6.13
Epoch 3897/4000, Loss_mean: 1.53,      Loss_min: 0.66, Loss_max: 2.23 
      Loss_mean_val: 15.76, Loss_min_val: 8.37,      Loss_max_val: 26.2 
      Loss_mean_trainval: 6.28
Epoch 3898/4000, Loss_mean: 1.54,      Loss_min: 0.81, Loss_max: 2.24 
      Loss_mean_val: 16.02, Loss_min_val: 7.96,      Loss_max_val: 30.46 
      Loss_mean_trainval: 6.37
Epoch 3899/4000, Loss_mean: 1.53,      Loss_min: 0.67, Loss_max: 2.26 
      Loss_mean_val: 15.77, Loss_min_val: 8.43,      Loss_max_val: 34.23 
      Loss_mean_trainval: 6.28
Epoch 3900/4000, Loss_mean: 1.58,      Loss_min: 0.88, Loss_max: 2.19 
      Loss_mean_val: 16.95, Loss_min_val: 7.54,   

Epoch 3942/4000, Loss_mean: 1.96,      Loss_min: 0.75, Loss_max: 2.79 
      Loss_mean_val: 15.66, Loss_min_val: 7.77,      Loss_max_val: 25.81 
      Loss_mean_trainval: 6.53
Epoch 3943/4000, Loss_mean: 1.98,      Loss_min: 0.9, Loss_max: 3.48 
      Loss_mean_val: 16.46, Loss_min_val: 4.48,      Loss_max_val: 40.06 
      Loss_mean_trainval: 6.81
Epoch 3944/4000, Loss_mean: 1.97,      Loss_min: 0.96, Loss_max: 3.29 
      Loss_mean_val: 16.36, Loss_min_val: 6.6,      Loss_max_val: 37.27 
      Loss_mean_trainval: 6.77
Epoch 3945/4000, Loss_mean: 1.95,      Loss_min: 0.75, Loss_max: 2.84 
      Loss_mean_val: 16.29, Loss_min_val: 6.22,      Loss_max_val: 35.5 
      Loss_mean_trainval: 6.73
Epoch 3946/4000, Loss_mean: 1.8,      Loss_min: 0.86, Loss_max: 3.15 
      Loss_mean_val: 16.41, Loss_min_val: 7.89,      Loss_max_val: 42.03 
      Loss_mean_trainval: 6.67
Epoch 3947/4000, Loss_mean: 1.84,      Loss_min: 0.76, Loss_max: 3.15 
      Loss_mean_val: 16.14, Loss_min_val: 8.41,      

Epoch 3989/4000, Loss_mean: 1.82,      Loss_min: 0.78, Loss_max: 2.92 
      Loss_mean_val: 15.6, Loss_min_val: 8.81,      Loss_max_val: 29.22 
      Loss_mean_trainval: 6.42
Epoch 3990/4000, Loss_mean: 1.97,      Loss_min: 0.82, Loss_max: 3.53 
      Loss_mean_val: 16.51, Loss_min_val: 6.9,      Loss_max_val: 31.81 
      Loss_mean_trainval: 6.82
Epoch 3991/4000, Loss_mean: 1.74,      Loss_min: 0.54, Loss_max: 3.1 
      Loss_mean_val: 15.13, Loss_min_val: 7.02,      Loss_max_val: 27.43 
      Loss_mean_trainval: 6.21
Epoch 3992/4000, Loss_mean: 1.63,      Loss_min: 0.76, Loss_max: 2.24 
      Loss_mean_val: 16.1, Loss_min_val: 8.43,      Loss_max_val: 24.18 
      Loss_mean_trainval: 6.46
Epoch 3993/4000, Loss_mean: 1.6,      Loss_min: 0.78, Loss_max: 2.85 
      Loss_mean_val: 16.15, Loss_min_val: 7.19,      Loss_max_val: 34.2 
      Loss_mean_trainval: 6.45
Epoch 3994/4000, Loss_mean: 1.57,      Loss_min: 0.74, Loss_max: 2.23 
      Loss_mean_val: 15.78, Loss_min_val: 5.48,      Lo